In [ ]:
# default_exp core

In [ ]:
#hide
# from google.colab import drive
# drive.mount("/content/drive")
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Coding/ModelAssistedLabel


# Configuration
Define necesasry default values and allow for the cloning of the yolov5 repo from Ultralytics

In [ ]:
# export
class Defaults:
  """
  Helps keep a DRY principle across this project. The split ratio was defined based
  on information found on Roboflow. `self.trainer_template` is pulled from the
  YOLOv5 tutorial, as is the code within `prepare_YOLOv5`
  """

  def __init__(self):
    self.root = "/content/drive/MyDrive/Coding/ModelAssistedLabel/"
    self.split_ratio = {
              "train": .7,
              "valid": .2,
              "test": .1
            }
    self.data_yaml = "\n".join(["train: ../train/images",
    "val: ../valid/images",
    "",
    "nc: 10",
    "names: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']"])
    self.resource_map = {"images": ".jpg", "labels": ".txt"}

    self.trainer_template = f"""# parameters
    nc: {10}  # number of classes
    depth_multiple: 0.33  # model depth multiple
    width_multiple: 0.50  # layer channel multiple

    # anchors
    anchors:
      - [10,13, 16,30, 33,23]  # P3/8
      - [30,61, 62,45, 59,119]  # P4/16
      - [116,90, 156,198, 373,326]  # P5/32

    # YOLOv5 backbone
    backbone:
      # [from, number, module, args]
      [[-1, 1, Focus, [64, 3]],  # 0-P1/2
      [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
      [-1, 3, BottleneckCSP, [128]],
      [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
      [-1, 9, BottleneckCSP, [256]],
      [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
      [-1, 9, BottleneckCSP, [512]],
      [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
      [-1, 1, SPP, [1024, [5, 9, 13]]],
      [-1, 3, BottleneckCSP, [1024, False]],  # 9
      ]

    # YOLOv5 head
    head:
      [[-1, 1, Conv, [512, 1, 1]],
      [-1, 1, nn.Upsample, [None, 2, 'nearest']],
      [[-1, 6], 1, Concat, [1]],  # cat backbone P4
      [-1, 3, BottleneckCSP, [512, False]],  # 13

      [-1, 1, Conv, [256, 1, 1]],
      [-1, 1, nn.Upsample, [None, 2, 'nearest']],
      [[-1, 4], 1, Concat, [1]],  # cat backbone P3
      [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

      [-1, 1, Conv, [256, 3, 2]],
      [[-1, 14], 1, Concat, [1]],  # cat head P4
      [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

      [-1, 1, Conv, [512, 3, 2]],
      [[-1, 10], 1, Concat, [1]],  # cat head P5
      [-1, 3, BottleneckCSP, [1024, 
      False]],  # 23 (P5/32-large)

      [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
      ]"""

  def prepare_YOLOv5():
    """
    * Clone repository if the YOLOv5 directory does not exist.
    * Install requirements.txt
    * Check that GPU is enabled.
    """
    # safety for re-executions
    if not os.path.exists("yolov5"):
      # clone YOLOv5 and reset to a specific git checkpoint that has been verified working
      os.system("git clone https://github.com/ultralytics/yolov5")  # clone repo
      os.system("git reset --hard 68211f72c99915a15855f7b99bf5d93f5631330f") # standardize models

    # enter the yolov5 directory
    os.chdir("yolov5")

    # install dependencies as necessary
    os.system("pip install -qr requirements.txt")  # install dependencies (ignore errors)
    import torch

    from IPython.display import Image, clear_output  # to display images
    # from utils.google_utils import gdrive_download  # to download models/datasets

    clear_output()

    if torch.cuda.is_available():
      print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0)))
    else:
      raise Exception("You need to enable your GPU access to this runtime environment")

    # return to parent directory
    os.chdir("..")

  #   calls = {}
  #   calls["nbdev"] = (["pip install nbdev"])
  #   calls["lib/docs"] = (["nbdev_build_lib", "nbdev_build_docs"])
  #   self.calls = calls

  # def call_all(self, arr):
  #   for x in arr:
  #     os.system(x)

  # def nbdev(self):
  #   for x in self.calls:
  #     self.call_all(calls[x])

**Default Values**
```
* root 
  --> parent folder of YOLOv5 repo

* resource_map
  --> defines images as ".jpg" and labels as ".txt"
* split_ratio 
  --> by default, 70/20/10 split of train/valid/test.

* data_yaml 
  --> from YOLOv5 repo
* trainer template 
  --> YOLOv5 repo
```


In [ ]:
print("\n".join(["Defined defaults:"]+[f"  * {x}" for x in Defaults().__dict__.keys()]))

Defined defaults:
  * root
  * split_ratio
  * data_yaml
  * resource_map
  * trainer_template
